# Mean Average Precision

* mAP

Mean Average Precision **evaluates** object detection models. It evaluates how well the model detects objects.
<!-- <!-- TODO read mean average percision paper and how its implement  -->
- It is typically done one the validation dataset.

### Steps In mAP


1. Get all bounding boxes predictions on our TEST set

    <img src="./ref_imgs/mAP/mAP_01.png" alt="Description" width="500"/>
    
    1. The green bboxes are the target boxes and the red are the predicted bounding boxes.

        <img src="./ref_imgs/mAP/mAP_02.png" alt="Description" width="400"/>
        
        FP = <span class="false-positive-color">False positives</span>, TP = <span class="true-positive-color">True positives</span>

        <img src="./ref_imgs/mAP/mAP_03.png" alt="Description" width="400"/>

        For this image we have a 0.9 confidence score which is a high IOU thats greater than 0.5 so thats a <span class="true-positive-color">True positive</span>.
    </br>
2. Get all the scoreces and TP/FP of all the predicted bounding boxes from each image, (i.e the dog pink background, the raincoat dogs, and the little dog yellow background ) 

    <img src="./ref_imgs/mAP/mAP_04.png" alt="Description" width="400"/>

    1. sort this table by descending confidence score

    <img src="./ref_imgs/mAP/mAP_05.png" alt="Description" width="400"/>

    </br>
3. A little bit about <span class="precision-text">Precision</span>  and <span class="recall-text">Recall</span> 

    <img src="./ref_imgs/mAP/mAP_06.png" alt="Description" width="700"/>

    </br>
    </br>

    1. <span class="true-positive-color">True positives</span> are predicted bboxes that share an IOU > 0.5 with a Target bounding box, and the <span class="false-positive-color">False positives</span> are the ones that doesn't.
    2. <span class="false-positive-color">False positives</span> means the model didn't predict a bounding box, but should've, True negatives is when the model predicts a bounding box but there is no target bounding box.
    3. <span class="precision-text">Precision</span>: take the <span class="true-positive-color">True positives</span> and divide by the <span class="true-positive-color">True positives</span> + <span class="false-positive-color">False positives</span>. So its trying to solve => <span class="highlight-text">(Of all bounding box predictions, what fraction was actually correct?)</span>
    4. <span class="recall-text">Recall</span>: take the <span class="true-positive-color">True positives</span> and divide it by the <span class="true-positive-color">True positives</span> +  <span class="false-positive-color">False positives</span>. So Recall is trying to => <span class="highlight-text">(Of all target bounding box, what fraction did we correctly detect?)</span>
    5. There's a battle between <span class="precision-text">Precision</span> and <span class="recall-text">Recall</span>! Different applications may priotitze <span class="recall-text">Recall</span> and others Precisions, for example self-driving cars prioritize not missing any pedestrains, so they would value having a high <span class="recall-text">Recall</span> more than <span class="precision-text">Precision</span>.
    </br>
    </br>
4. Calculate the <span class="precision-text">Precision</span> and <span class="recall-text">Recall</span> as we go through all predicted output bounding boxes from the model.

    <img src="./ref_imgs/mAP/mAP_07.png" alt="Description" width="500"/>

    1. The first (Image 3 0.9 confidence) is a True positive, so the <span class="precision-text">Precision</span> is 1/1 <span class="highlight-text">(because our prediction so far has been 100% accurate)</span>, and the <span class="recall-text">Recall</span> is 1/4 <span class="highlight-text">(because we have only gone through and correctlly predicted 1 out of 4 target bounding boxes).</span>
    2. The second row is a False positive, so the <span class="precision-text">Precision</span> is going to be 1/2 <span class="highlight-text">because we have only correctly predicted one out two</span>. And the <span class="recall-text">Recall</span> is going to stay the same because <span class="highlight-text"> we haven't predicted any more correctly.</span>.
    3. The third row is a TP so <span class="precision-text">Precision</span> 2/3 and <span class="recall-text">Recall</span> 2/4.
    4. etc.. for the remaining rows/predicted bounding boxes.
    </br>
5. Plot The Recall-Precision graph. 

    <img src="./ref_imgs/mAP/mAP_08.png" alt="Description" width="600"/>

    1. Calculate Area Under PR Curve.

    <img src="./ref_imgs/mAP/mAP_09.png" alt="Description" width="600"/>

        Take the area which is Dog AP = 0.533.

    2. We have been trying to predict dogs in this example dateset. But when we are predicting for more than one class, we need to calculate for all classes. </br>Example dataset cats and dogs: </br>
        Cat AP = 0.74 </br>
        Dog AP = 0.533</br>
        <span class="highlight-text">mAP = ( 0.533 + 0.74 ) / 2 = 0.63645</span>
    </br>
6. All this was calcualted given a specific IOU threshold of 0.5, we need to redo all computations for many IOU's, example: [0.5, 0.55, 0.6, .., 0.95]. Then average this and this will be out final result. This is what is meant by <span class="highlight-text">mAP@0.5:0.05:0.95</span>.


<!-- <span class="highlight-text"></span> -->
<!-- <span class="precision-text">Precision</span> -->
<!-- <span class="recall-text">Recall</span> -->
<!-- <span class="false-positive-color">False positives</span> -->
<!-- <span class="true-positive-color">False positives</span> -->

<!-- CSS BELOW -->
<style>
    .false-positive-color{color: #F87575}
    .true-positive-color{color: #5C95FF}
    .highlight-text{color:orange; font-weight:400}
    .recall-text{color:#16BAC5}
    .precision-text{color:green}


</style>

### Implement In Pytorch

In [7]:
import torch
from collections import Counter
from utils.IOU import intersection_over_union


In [ ]:
def mean_average_precision(
    pred_bboxes,
    true_bboxes,
    iou_threshold=0.5,
    box_format="corners",
    num_classes = 20
):
    """
    Compute Mean Average Precision (mAP), this function isn't the most effiecent implementation of mAP
    
    NOTE: this implementation is only for a single IOU = 0.5, if we want to compute this for multiple IOU's = [0.5, 0.55, 0.9, etcc..] then call this function for for each IOU, store the result in a list and then take the average from that list
    
    Parameters
    -----------
        pred_bboxes (list): all predicted output from the models for all images in the test set. [ [train_idx, class_pred, prob_score, x1, y1, x2, y2], [etc..], etc..], train_idx = what image is it.
        true_bboxes (list): all true bounding boxes from the images in the test set. Almost identical to the structure of pred_bboxes.
        iou_threshold (float): the threshold to remove predicted bounding boxes if it doesn't cover the true bounding boxes enough.
        box_format (string): from where to draw the predicted bounding boxes, from corners or mid-point
        num_classes (int): how many classes are we predicting [example: dogs, cats, cars, etc..]
    
    
    Return
    ------
        mAP
    """
    # list storing all AP for respective classes
    average_precision = []
    epsilon = 1e-6# for numerical stability
    
    # we need to calculate for each class
    for c in range(num_classes):
        detections = [] 
        ground_truths = []
        
        
        # Go through all predictions
        for detect in pred_bboxes:
            if detect[1] == c: #[1] is the class_pred, if its the same as the class were currently looking at then append
                detections.append(detect) 
        #Go through all targets
        for t_b in true_bboxes:
            if t_b[1] == c: 
                ground_truths.append(t_b)
        
        # [0] is the image index in the dataset
        # ex: image 0 has 3 bounding boxes
        # ex: image 1 has 5 bounding boxes
        # amount_bboxes = { 0 : 3, 1 : 5}
        amount_bboxes = Counter([gt[0] for gt in ground_truths])
        
        # this is to keep track of the bounding boxes that we have seen
        for key, val in amount_bboxes.items():
            # turns the 3 here counter = { 0 : 3, 1 : 5} into a tensor of 3 elements of zero's
            # amount_bboxes = { 0 : torch.tensor([0,0,0]), 1 : torch.tensor([0,0,0,0,0])}
            amount_bboxes[key] = torch.zeros(val)
            
        
        detections.sort(key=lambda x: x[2], reverse=True) # sort over the prob_scores, this is the 2.1 in the markdown
        TP = torch.zeros((len(detections)))# True positives
        FP = torch.zeros((len(detections)))# False positives
        total_true_bboxes = len(ground_truths)
        
        # If none exists for this class then we can safely skip
        if total_true_bboxes == 0:
            continue
        
        for detect_idx, detect in enumerate(detections):
            ground_truth_img = [
                bbox for bbox in ground_truths if bbox[0] == detect[0] # we are only taking the ground truths that have the same index as out detected/ predicted
            ]
            
            num_gts = len(ground_truth_img) # number of ground truths
            best_iou = 0
            
            # go through all the ground truth bounding box for this image
            for idx, gt in enumerate(ground_truth_img):
                iou = intersection_over_union(
                    torch.tensor(detect[3:]), # only sending the coordinates of the bbox [x1, y1, x2, y2]
                    torch.tensor(gt[3:]), # only sending the coordinates of the bbox [x1, y1, x2, y2]
                    box_format=box_format,
                )
                
                if iou > best_iou:
                    best_iou = iou
                    best_gt_idx = idx
            
            if best_iou > iou_threshold: # means that this prediction was correct 
                # if it eqauls to 0 than we haven't looked at that bbox yet
                if amount_bboxes[detect[0]][best_gt_idx] == 0: # but we also need to check that this bbox hasn't been looked at already
                    TP[detect_idx] = 1
                    amount_bboxes[detect[0]][best_gt_idx] = 1 # since we've covered this bbox, we set this to 1
                else:
                    FP[detect_idx] = 1
            else: # if IOU is lower then the detection is a false positive
                FP[detect_idx] = 1
        
        ### Precision and Recall 
        # [1, 1, 0, 1] --> [1, 2, 2, 3, 3] # the first list the 1 means correctly predicted and the 0 means incorrectly predicted
        TP_cumsum = torch.cumsum(TP, dim=0)
        
        FP_cumsum = torch.cumsum(FP, dim=0)
        
        recalls = TP_cumsum / (total_true_bboxes + epsilon)
        
        precision = torch.divide(TP_cumsum, (TP_cumsum + FP_cumsum + epsilon))
        
        precision = torch.cat((torch.tensor([1]), precision)) # we need to add a 1, because we want to start at the point zero one torch.tensor([1]) because precision is the y-axis on the graph
        recalls = torch.cat((torch.tensor([0]), recalls)) # we need to add a 1, because we want to start at the point zero one torch.tensor([0]) because recalls is the x-axis on the graph
        
        average_precision.append(torch.trapz(precision, recalls))  # this is 5.1 calculate the area under PR curve
        
    return sum(average_precision) / len(average_precision)